In [ ]:
from pathlib import Path
import pickle
import json
import pandas as pd
import numpy as np
from naslib.data.api import get_nasbench201_robustness_api
from naslib.data.nas_bench_201_robustness.robustness_data import RobustnessDataset
from naslib.utils.io import read_json

In [ ]:
home_dir = Path("/Volumes/NO NAME/results/test_mutation/nasbench201")

datasets = ["cifar10", "cifar100", "ImageNet16-120"]
algos = [
    "acq=its/num_to_mutate=2/num_init=10/bananas__ensemble_mlp__gaussian__num_quantiles=10",
    "acq=its/num_to_mutate=2/num_init=10/bananas__ensemble_mlp__CP_split__train_cal_split=05__num_quantiles=10",
    "acq=its/num_to_mutate=2/num_init=10/bananas__ensemble_mlp__CP_cv__train_cal_split=3__num_quantiles=10",
    "acq=its/num_to_mutate=2/num_init=10/bananas__ensemble_mlp__CP_bootstrap__num_ensemble=5__num_quantiles=10_absresidual",
    "acq=its/num_to_mutate=2/num_init=10/bananas__quantile__CP_split__train_cal_split=05__num_quantiles=10",
    "acq=ei/num_to_mutate=2/num_init=10/bananas__quantile__CP_cv__train_cal_split=3__num_quantiles=10"
]

### Get the string representation of the best performing architecture 

In [ ]:
def get_str_repr_best_arch(result_folder: Path):
    with open(result_folder / "search_log.pt", "rb") as f:
        history, obs_and_dist = pickle.load(f)

        # get the best architecture
        val_scores, _ = list(zip(*obs_and_dist))
        best_arch_idx = val_scores.index(max(val_scores))
        arch_str = history[best_arch_idx][1]

    with open(result_folder / "arch_name.txt", "w") as f:
        f.write(arch_str)

In [ ]:
# for data in datasets:
#     for algo in algos:
#         algo_folder = home_dir / data / algo
#         # get all trial folders
#         matches = list(algo_folder.rglob(pattern=f"./seed=*"))
#         for single_result_folder in matches:
#             get_str_repr_best_arch(single_result_folder)

### Get the robustness accuracy

In [ ]:
ROBUST_CLIENT = RobustnessDataset(path="./naslib/data/nas_bench_201_robustness")

#### Get test accuracy functions

In [ ]:
def get_clean_test_accuracy(arch_str: str, dataset: str) -> dict[str, float]:
    data_api = get_nasbench201_robustness_api(dataset=dataset, key="clean")
    arch_id = ROBUST_CLIENT.string_to_id(s=arch_str)
    return {"clean": data_api[arch_id]}

In [ ]:
def get_aa_apgd_ce_accuracy(arch_str: str, dataset: str) -> dict[str, float]:
    data_api = get_nasbench201_robustness_api(dataset=dataset, key="aa_apgd-ce@Linf")
    attack_levels = ["0.1", "0.5", "1", "2", "3", "4", "8"]

    arch_id = ROBUST_CLIENT.string_to_id(s=arch_str)
    accuracies = data_api[arch_id]
    assert len(accuracies) == len(attack_levels)

    return {f"aa_apgd-ce@Linf_{level}": accuracies[idx] for idx, level in enumerate(attack_levels)}

In [ ]:
def get_aa_square_accuracy(arch_str: str, dataset: str) -> dict[str, float]:
    data_api = get_nasbench201_robustness_api(dataset=dataset, key="aa_square@Linf")
    attack_levels = ["0.1", "0.5", "1", "2", "3", "4", "8"]

    arch_id = ROBUST_CLIENT.string_to_id(s=arch_str)
    accuracies = data_api[arch_id]
    assert len(accuracies) == len(attack_levels)

    return {f"aa_square@Linf_{level}": accuracies[idx] for idx, level in enumerate(attack_levels)}

In [ ]:
def get_fgsm_accuracy(arch_str: str, dataset: str) -> dict[str, float]:
    data_api = get_nasbench201_robustness_api(dataset=dataset, key="fgsm@Linf")
    attack_levels = ["0.1", "0.5", "1", "2", "3", "4", "5", "6", "7", "8", "255"]

    arch_id = ROBUST_CLIENT.string_to_id(s=arch_str)
    accuracies = data_api[arch_id]
    assert len(accuracies) == len(attack_levels)

    return {f"fgsm@Linf_{level}": accuracies[idx] for idx, level in enumerate(attack_levels)}

In [ ]:
def get_pgd_accuracy(arch_str: str, dataset: str) -> dict[str, float]:
    data_api = get_nasbench201_robustness_api(dataset=dataset, key="pgd@Linf")
    attack_levels = ["0.1", "0.5", "1", "2", "3", "4", "8"]

    arch_id = ROBUST_CLIENT.string_to_id(s=arch_str)
    accuracies = data_api[arch_id]
    assert len(accuracies) == len(attack_levels)

    return {f"pgd@Linf_{level}": accuracies[idx] for idx, level in enumerate(attack_levels)}

#### Build robust accuracy result files

In [ ]:
def build_robustness_result_log(result_folder: Path, dataset: str):
    with open(result_folder / "arch_name.txt", "r") as f:
        arch_str = f.read()

    result = {}
    result.update(get_clean_test_accuracy(arch_str=arch_str, dataset=dataset))
    result.update(get_aa_apgd_ce_accuracy(arch_str=arch_str, dataset=dataset))
    result.update(get_aa_square_accuracy(arch_str=arch_str, dataset=dataset))
    result.update(get_fgsm_accuracy(arch_str=arch_str, dataset=dataset))
    result.update(get_pgd_accuracy(arch_str=arch_str, dataset=dataset))

    with open(result_folder / "robustness.json", "w") as f:
        json.dump(result, fp=f)

In [ ]:
# for data in datasets:
#     for algo in algos:
#         algo_folder = home_dir / data / algo
#         # get all trial folders
#         print(f"running {algo_folder}")
#         matches = list(algo_folder.rglob(pattern=f"./seed=*"))
#         for single_result_folder in matches:
#             try:
#                 build_robustness_result_log(result_folder=single_result_folder, dataset=data)
#             except KeyError:
#                 print(f"{single_result_folder.name} not found in robustness dataset") 

### Analysis

In [ ]:
ROBUSTNESS_FILENAME = "robustness.json"

def collect_json_info_all_seeds(folder: Path, filename: str = ROBUSTNESS_FILENAME):
    matches = list(folder.rglob(pattern=f"./seed=*"))

    result = {}
    for p in matches:
        if not (p / filename).exists():
            result[p.name] = pd.DataFrame(index=[0])
        else:
            result[p.name] = pd.DataFrame(read_json(p / filename), index=[0])
    return pd.concat(result, axis=0).droplevel(level=1)
    